In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Finite differences for the heat equation

## Finite-difference formulation

The 1D heat equation for diffusion (conduction) only and a constant thermal conductivity is

\begin{equation}
  \rho C_p \frac{\partial T}{\partial t} = k \frac{\partial^2 T}{\partial x^2}
\end{equation}

The second-order spatial derivative of temperature can be approximated with finite differences as

\begin{align}
  \frac{\partial^2 T}{\partial x^2}\Big\vert_{x=x_1}^{t=t_1} 
&\approx \left(\frac{\partial T}{\partial x}\Big\vert_{x=x_1+0.5\Delta x}^{t=t_1} - \frac{\partial T}{\partial x}\Big   \vert_{x=x_1-0.5\Delta x}^{t=t_1}\right) / \Delta x \newline
&\approx \left(\frac{T_{x=x_1+\Delta x}^{t=t_1} - T_{x=x_1}^{t=t_1}}{\Delta x} - \frac{T_{x=x_1}^{t=t_1} -   T_{x=x_1-\Delta x}^{t=t_1}}{\Delta x}\right) / \Delta x
  = \frac{T_{x=x_1+\Delta x}^{t=t_1} - 2T_{x=x_1}^{t=t_1} + T_{x=x_1 - \Delta x}^{t=t_1}}{(\Delta x)^2}
\end{align}

Since $x_1 - \Delta x = x_0$ and $x_1 + \Delta x = x_2$, we get

\begin{equation}
  \frac{\partial^2 T}{\partial x^2}\Big\vert_{x=x_1}^{t=t_1} \approx \frac{T_{x=x_2}^{t=t_1} - 2T_{x=x_1}^{t=t_1} + T_{x=x_0}^{t=t_1}}{(\Delta x)^2} = \frac{T_{2}^{1} - 2T_{1}^{1} + T_{0}^{1}}{(\Delta x)^2}
\end{equation}

where the last formula uses the short hand notation for indices. Note that here the superscripts do not mean "to the power" but are indices for time steps.

Using the same short hand notation, we can approximate the time derivative of the temperature as

\begin{equation}
  \frac{\partial T}{\partial t}\Big\vert_{x=x_1}^{t=t_1} \approx \frac{T_1^2-T_1^1}{\Delta t}
\end{equation}

These approximations can be generalized as

\begin{align}
  \frac{\partial^2 T}{\partial x^2}\Big\vert_{x=x_i}^{t=t_q} &\approx \frac{T_{i+1}^{q} - 2T_{i}^{q} + T_{i-1}^{q}}{(\Delta x)^2} \newline
  \frac{\partial T}{\partial t}\Big\vert_{x=x_i}^{t=t_q} &\approx \frac{T_i^{q+1}-T_i^q}{\Delta t}
\end{align}

And thus the full finite differences approximation of the heat equation can be written as

\begin{equation}
  \rho C_p \frac{T_i^{q+1}-T_i^q}{\Delta t} = k \frac{T_{i+1}^{q} - 2T_{i}^{q} + T_{i-1}^{q}}{(\Delta x)^2}
\end{equation}

This applies to *any* $q$ and $i$. If we know $T$ everywhere at time $t_q$, we can calculate $T$ at time $t_{q+1} = t_q + \Delta t$.

### Exercise - The implicit form?

- How would the implicit (backward difference in time) approximation look like for the heat equation?
    - Recall that in the implicit approximation we used the values from the next time step to approximate the derivative.
    - What is the problem in calculating $T_i^q$ using the implicit approximation?

## Boundary and initial conditions

To start calculating the values of $T$, we need the *initial* values at some time $t_0$. This temperature field is the so called *initial condition* for our problem. It is a type of boundary condition that applies to a time varying field.

(Spatial) boundary conditions are needed because when applying finite differences to a differential equation (in space), we need the surrounding values, such as $T_{i+1}$ and $T_{i-1}$ in the case of heat equation, to calculate the next value of $T_{i}$. These values are not available if we are near the geometrical boundary of the problem.

## Cooling of an intrusion

Next we will use finite differences to model the cooling of a (very large) magma intrusion within the crust. The crust is limited by the upper surface and the crust-mantle boundary at the bottom. It is 35 km thick and intruded by a horizontal, granitic magma intrusion (sill) that extends from 5 km to 10 km depth. Our task is to calculate the temperature distribution in the crust after 100 kyrs.

Our strategy for solving the problem in Python is the following:

0. Draw a picture!
1. Decide on model geometry and physical parameters
    - One-dimensional
    - Homogeneous physical parameters
    - $\Rightarrow$ we can use the equation above
2. Set physical parameters ($\rho$, $C_p$, $k$)
3. Set geometrical parameters ($L$, $\Delta x$ $\Rightarrow$ `nx`)
4. Set total run time and time step size ($t_{total}$, $\Delta t$ $\Rightarrow$ `nt`)
5. Decide and set boundary condition values $T_0$ and $T_{nx-1}$
6. Create two arrays that hold the temperature, size `nx` elements
    - `Tnew` for $T^{q+1}$
    - `Tprev` for $T^{q}$
7. Create an array for grid point locations ($x$ coordinates), size `nx`
8. Set `Tprev` to hold the initial temperature field
    - Linear geotherm throughout the crust, constant temperature within the sill
9. Start the loop that iterates over all the time steps. Within the time step:
    - Set the temperature at the boundaries, according to boundary conditions, at `Tnew[0]` and `Tnew[nx-1]`
    - Calculate remaining (internal grid points) within `Tnew` based on values of `Tprev`
    - Prepare for next time step: Copy all values of `Tnew` to `Tprev`


### Exercise - Coding your solution, considering your results

- Use the outline above to start implementing the code. Once the code seems to work, experiment with it:
    - How does changing values of `Cp` and `rho` affect the results?
    - Give `dt` values that are close to the critical time step. How do the results change?
    - Can the code handle intrusions that are only 500 m in width?
- What have we silently assumed in the implementation of the code? For example, can we model the problem in simplified 1D world? What kind of physical processes that are likely to be present may we have omitted?

In [ ]:
### ANSWER ###

import numpy as np
import matplotlib.pyplot as plt

yr2sec = 60*60*24*365.25  # num of seconds in a year

# Set material properties
rho = 2900   # kg m^-3
Cp = 900     # J kg^-1 K^-1
k = 2.5      # W m^-1 K^-1

# Set geometry and dimensions
L = 35e3         # m
nx = 30          # -
dx = L / (nx-1)  # m
print("dx is", dx, "m")

silltop = 5e3    # m
sillbott = 10e3  # m

# Set total time and time step
t_total = 0.1e6 * yr2sec          # s
dt = 1e4 * yr2sec                 # s
nt = int(np.floor(t_total / dt))  # -
print("Calculating", nt, "time steps")

# Check for a stable solution
stable = ((k/(rho*Cp)) * dt) / (dx**2)
if stable > 0.5:
    print('WARNING: Unstable solution expected!')

# Set boundary temperature values and intrusion temperature
Tsurf = 0.0        # deg C
Tmoho = 600.0      # deg C
Tintrusion = 700.0 # deg C

# Create arrays to hold temperature fields
Tnew = np.zeros(nx)
Tprev = np.zeros(nx)

# Create coordinates of the grid points
x = np.linspace(0, L, nx)

# Generate initial temperature field
for ix in range(nx):
    if (x[ix] < sillbott) and (x[ix] > silltop):
        Tprev[ix] = Tintrusion
    else:
        Tprev[ix] = x[ix] * (Tmoho - Tsurf) / L

# Plot initial temperature field
plt.plot(Tprev, -x, '.-', label='T initial')
plt.legend()
#plt.show()

# Start the loop over time steps
curtime = 0
while curtime < t_total:
    curtime = curtime + dt
    
    # boundary conditions
    Tnew[0] = Tsurf
    Tnew[nx-1] = Tmoho
    
    # internal grid points
    for ix in range(1, nx-1):
        Tnew[ix] = (k * (Tprev[ix-1] - 2.0*Tprev[ix] + Tprev[ix+1]) / dx**2) * dt / (rho * Cp) + Tprev[ix]
        
    # copy Tnew to Tprev
    Tprev[:] = Tnew[:]
    

# Plot the final temperature field
plt.plot(Tnew, -x, '.-', label='T final')
plt.legend()
plt.show()